In [8]:
from src.config import LocalConfig, ColabConfig, is_colab

config = ColabConfig() if is_colab() else LocalConfig()

print("Using configuration:", type(config).__name__)
print("Base directory:", config.BASE_DIR)

config.ensure_dirs()

Using configuration: LocalConfig
Base directory: /mnt/c/dev/ml/rag-qa
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/.hf_cache
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data/train
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data/validation
✅ Ensured directory exists: /mnt/c/dev/ml/rag-qa/data/test


In [9]:
from src.config import Config
from src.compute_embeddings import compute_embeddings
import shutil
import os

chunk_settings = [
    (160, 40),
    (200, 50),
    (240, 60),
]

MAX_SHARDS = 2  # je Split
DEV_RUNS = []

FAISS_INDEX_FILE = config.FAISS_INDEX_FILE
EMBEDDINGS_FILE = config.EMBEDDINGS_FILE
PASSAGES_FILE = config.PASSAGES_FILE

for idx, (chunk_tokens, chunk_overlap) in enumerate(chunk_settings, start=1):
    config.FAISS_INDEX_FILE = f"{FAISS_INDEX_FILE}.run{idx}"
    config.EMBEDDINGS_FILE = f"{EMBEDDINGS_FILE}.run{idx}"
    config.PASSAGES_FILE = f"{PASSAGES_FILE}.run{idx}"
    print(config.FAISS_INDEX_FILE)
    print(config.PASSAGES_FILE)
    print(config.EMBEDDINGS_FILE)

    corpus, emb = compute_embeddings(
        config,
        force_recompute=True,
        recompute_passages=True,
        data_dirs=[config.TRAIN_DIR, config.VAL_DIR],  # nur train+val für den Test
        max_shards_per_dir=MAX_SHARDS,
        chunk_tokens=chunk_tokens,
        chunk_overlap=chunk_overlap,
    )

    print(f"Run {idx} done. Built {len(corpus) if corpus else 0} passages.")
    DEV_RUNS.append((chunk_tokens, chunk_overlap))

print("\nRuns fertig. Starte nun evaluate_retrieve (DEV_LIMIT z.B. 200) pro Run, falls du die Artefakte getrennt hältst.")

/mnt/c/dev/ml/rag-qa/corpus_faiss.index.run1
/mnt/c/dev/ml/rag-qa/corpus_passages.pkl.run1
/mnt/c/dev/ml/rag-qa/corpus_embeddings_unique.pkl.run1
Embeddings not found or force_recompute=True, computing embeddings...


Loading shards from /mnt/c/dev/ml/rag-qa/data/train: 100%|██████████| 2/2 [00:00<00:00, 129.92it/s]
Loading shards from /mnt/c/dev/ml/rag-qa/data/validation: 100%|██████████| 2/2 [00:00<00:00, 151.48it/s]

Loaded dataset with 4000 examples.


Using device: cuda


Extracting passages: 100%|██████████| 4000/4000 [02:03<00:00, 32.29it/s]


...

Using device: cuda


Extracting passages: 100%|██████████| 4000/4000 [01:49<00:00, 36.45it/s]


Removed 96231 duplicate passages (151826 unique).
Saved passages to /mnt/c/dev/ml/rag-qa/corpus_passages.pkl.run3
Using device: cuda


Computing embeddings: 100%|██████████| 297/297 [03:23<00:00,  1.46it/s]


Saved embeddings to /mnt/c/dev/ml/rag-qa/corpus_embeddings_unique.pkl.run3
Saved FAISS index to /mnt/c/dev/ml/rag-qa/corpus_faiss.index.run3 (dim=384, n=151826)
Run 3 done. Built 151826 passages.

Runs fertig. Starte nun evaluate_retrieve (DEV_LIMIT z.B. 200) pro Run, falls du die Artefakte getrennt hältst.


In [13]:
from src.evaluate_retrieve import run_evaluation

config.EMBEDDINGS_FILE = f"corpus_embeddings_unique.pkl.run3"
config.PASSAGES_FILE = f"corpus_passages.pkl.run3"
config.FAISS_INDEX_FILE = f"corpus_faiss.index.run3"
run_evaluation(config=config)

corpus_faiss.index.run3
🔹 Loaded FAISS index with 151826 passages

=== 🔥 Evaluating TRAIN — first 1000 samples ===


Evaluating Recall: 100%|██████████| 1000/1000 [00:16<00:00, 60.04it/s]


Recall@1: 0.6460
Recall@3: 0.8440
Recall@5: 0.8840
Recall@7: 0.9080
Recall@10: 0.9230

=== 🔥 Evaluating VALIDATION — first 1000 samples ===


Evaluating Recall: 100%|██████████| 1000/1000 [00:14<00:00, 69.40it/s]


Recall@1: 0.5490
Recall@3: 0.7580
Recall@5: 0.8070
Recall@7: 0.8380
Recall@10: 0.8700

=== 🔥 Evaluating TEST — first 1000 samples ===


Evaluating Recall: 100%|██████████| 1000/1000 [00:15<00:00, 65.51it/s]

Recall@1: 0.3600
Recall@3: 0.5120
Recall@5: 0.5630
Recall@7: 0.6010
Recall@10: 0.6300
